In [3]:
import sqlite3

connection = sqlite3.connect(':memory')
cursor = connection.cursor()

In [13]:
from datetime import datetime, timedelta

from jellyfish.core import Client
from jellyfish.history_loader import get_sample_frame

client = Client()

pair_sym = 'BTCUSDT'
interval = '1h'
end_dt = datetime.now()
start_dt = end_dt - timedelta(hours=7)

candles = client.get_historical_klines(pair_sym, interval, str(start_dt), str(end_dt))
candles = [i[1:11] for i in candles]
candles[0]

['21235.84000000',
 '21285.00000000',
 '20722.04000000',
 '20737.81000000',
 '7499.50923000',
 1656338399999,
 '157267545.88894270',
 100848,
 '3458.55654000',
 '72536375.09945470']

In [14]:
cmd = """
    CREATE TABLE IF NOT EXISTS %s (
        open REAL,
        high REAL,
        low REAL,
        close REAL,
        volume REAL,
        timestamp INT PRIMARY KEY,
        quote_asset_volume REAL,
        num_of_trades INTEGER,
        taker_buy_asset_volume REAL,
        taker_sell_asset_volume REAL
    )
"""

cmd = ' '.join((cmd % pair_sym).split())
cursor.execute(cmd)

In [15]:
cmd = f'INSERT OR IGNORE INTO {pair_sym} VALUES ({",".join("?" for _ in candles[0])})'
cmd
cursor.executemany(cmd, candles)

In [16]:
for i in cursor.execute(f'SELECT * FROM {pair_sym}'):
    print(i)

(21235.84, 21285.0, 20722.04, 20737.81, 7499.50923, 1656338399999, 157267545.8889427, 100848, 3458.55654, 72536375.0994547)
(20737.82, 20845.37, 20590.0, 20820.51, 7174.28946, 1656341999999, 148537189.7972275, 99268, 3541.35965, 73321496.9913034)
(20820.5, 20851.51, 20675.11, 20754.73, 3241.02673, 1656345599999, 67306673.0624312, 53457, 1611.31093, 33460047.14791)
(20754.74, 20762.57, 20702.0, 20702.01, 531.23138, 1656349199999, 11011141.8259208, 10915, 267.01295, 5534586.9654252)


In [17]:
cmd = f'DELETE FROM {pair_sym}'
print(cmd)
cursor.execute(cmd)

DELETE FROM BTCUSDT
